Instalando pacotes que serão utilizados

In [12]:
%pip install pandas openpyxl mysql-connector-python sqlalchemy pymysql requests xlrd dash cryptography


Note: you may need to restart the kernel to use updated packages.


Fazendo as importações

In [13]:
import pandas as pd
from sqlalchemy import create_engine, text
import requests
import xlrd
import os
from dash import Dash, html, dcc, html, Input, Output, State, dash
import plotly.express as px
import pandas as pd





API para o download dos arquivos 

In [14]:

url_gdppc = "https://dataverse.nl/api/access/datafile/421302"

if requests.get(url_gdppc).status_code == 200:  
    with open("gdppc.xlsx", "wb") as f:
        f.write(requests.get(url_gdppc).content) 
    print("Arquivo baixado e salvo com sucesso!")
else:
    print("Erro ao baixar o arquivo:", requests.get(url_gdppc).status_code)


Arquivo baixado e salvo com sucesso!


In [15]:

url_gpd = "https://www.imf.org/external/datamapper//export/excel.php?indicator=d"

if requests.get(url_gpd).status_code == 200:  
    with open("gpd.xls", "wb") as f:
        f.write(requests.get(url_gpd).content)  
    print("Arquivo baixado e salvo com sucesso!")
else:
    print("Erro ao baixar o arquivo:", requests.get(url_gpd).status_code)
   

Arquivo baixado e salvo com sucesso!


Criando conexões com banco

                                CONEXÃO COM SQL SERVER (Caso queira utilizar MySQL ignore a proxima parte)

                                Para conexão local (localhost) não precisa trocar as informações das variaveis server e database

In [ ]:

server = "localhost"  
database = "master"  
db_name = "desafioUnaGabriel"  




# Criar engine para se conectar ao SQL Server
engine = create_engine(f"mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server")

# Criar o banco de dados se não existir
  # Substitua pelo nome do banco desejado

with engine.connect() as connection:
    connection.execution_options(isolation_level="AUTOCOMMIT").execute(text(f"IF NOT EXISTS (SELECT name FROM sys.databases WHERE name = '{db_name}') CREATE DATABASE {db_name}"))

# Conectar-se ao novo banco de dados
engine = create_engine(f"mssql+pyodbc://@{server}/{db_name}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server")

print("Banco foi criado com sucesso")

Banco foi criado com sucesso


                                CONEXÃO COM MYSQL (Caso deseje utilizar o SQL SERVER volte para a parte de cima e ignore essa)

                                Conexão padrão pelo root e no localhost, troque pela sua senha do MySQL server e caso desejar troque o nome da base na variavel database (Obs: verifique antes de trocar se não há uma base existente igual a essa ou o nome que colocar em database)

CONEXÃO COM MYSQL (OBS: coloque as informações do MySQL da sua maquina e garanta que o servidor esteja rodando para realizar a conexão)

In [ ]:
user = "root"
senha = ""
server = "localhost"
database = "desafioUnaGabriel"

engine = create_engine(f"mysql+pymysql://{user}:{senha}@{server}/") 

with engine.connect() as connection:
    connection.execution_options(isolation_level="AUTOCOMMIT").execute(text(f'CREATE DATABASE IF NOT EXISTS {database}')) 

engine = create_engine(f"mysql+pymysql://{user}:{senha}@{server}/{database}")

print('Banco foi criado com sucesso')



Banco foi criado com sucesso


Limpeza e exportação dos dados para o banco de dados

In [17]:
df_gdppc = pd.read_excel("gdppc.xlsx", sheet_name="Full data")
df_gdppc = df_gdppc.dropna() # remove nulos 
df_gdppc.to_excel("gdppc_clean.xlsx", index=False)# arquivo excel com dados limpos

#excluir o arquivo original
if os.path.exists("gdppc.xlsx"):
    os.remove("gdppc.xlsx")
    print("Arquivo excluído")
else:
    print("O arquivo não existe.")


Arquivo excluído


In [18]:
workbook = xlrd.open_workbook('gpd.xls', ignore_workbook_corruption=True) #leitura de arquivo xls
df_gpd = pd.read_excel(workbook)
df_gpd.info

<bound method DataFrame.info of     Gross public debt, percent of GDP (% of GDP)     1800     1801     1802  \
0                                            NaN      NaN      NaN      NaN   
1                                        Albania  no data  no data  no data   
2                                        Algeria  no data  no data  no data   
3                                         Angola  no data  no data  no data   
4                            Antigua and Barbuda  no data  no data  no data   
..                                           ...      ...      ...      ...   
149                                    Venezuela  no data  no data  no data   
150                                      Vietnam  no data  no data  no data   
151                                        Yemen  no data  no data  no data   
152                                          NaN      NaN      NaN      NaN   
153                                   ©IMF, 2024      NaN      NaN      NaN   

        1803     18

Limpeza e exportação dos dados para o banco de dados(gpd)

In [19]:
df_gpd = df_gpd.rename(columns={"Gross public debt, percent of GDP (% of GDP)" : "country"})
u_df = df_gpd.melt(id_vars=["country"], var_name="year", value_name="gpd")
df_gpd = u_df
df_gpd["gpd"].replace("no data", pd.NA, inplace=True)

df_gpd = df_gpd.dropna()

df_gpd.to_excel("gpd_clean.xlsx", index=False)
df_gpd.info
df_gpd = pd.read_excel("gpd_clean.xlsx", sheet_name="Sheet1")

if os.path.exists("gpd.xls"):
    os.remove("gpd.xls")
    print("Arquivo excluído")
else:
    print("O arquivo não existe.")

C:\Users\lilri\AppData\Local\Temp\ipykernel_6720\1891571809.py:4: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





Arquivo excluído


In [20]:
df_gdppc.to_sql("gdppc", con=engine, if_exists="replace", index=False)
df_gpd.to_sql("gpd", con=engine, if_exists="replace", index=False)

print("Dados Exportados!")

Dados Exportados!


In [9]:
#Teste para ver como os dados estão dentro do banco

query_gdppc = "SELECT * FROM gdppc"  
df_teste_gdppc = pd.read_sql(query_gdppc, engine)

print(df_teste_gdppc)


      countrycode      country                     region  year        gdppc  \
0             AFG  Afghanistan  South and South East Asia  1950  1156.000000   
1             AFG  Afghanistan  South and South East Asia  1951  1170.000000   
2             AFG  Afghanistan  South and South East Asia  1952  1189.000000   
3             AFG  Afghanistan  South and South East Asia  1953  1240.000000   
4             AFG  Afghanistan  South and South East Asia  1954  1245.000000   
...           ...          ...                        ...   ...          ...   
15956         ZWE     Zimbabwe         Sub Saharan Africa  2018  1900.199191   
15957         ZWE     Zimbabwe         Sub Saharan Africa  2019  1753.024449   
15958         ZWE     Zimbabwe         Sub Saharan Africa  2020  1585.972804   
15959         ZWE     Zimbabwe         Sub Saharan Africa  2021  1687.253154   
15960         ZWE     Zimbabwe         Sub Saharan Africa  2022  1703.529373   

            pop  
0       8150.00  
1  

In [10]:
#Teste para ver como os dados estão dentro do banco

query_gpd = "SELECT * FROM gpd"  
df_teste_gpd = pd.read_sql(query_gpd, engine)

print(df_teste_gpd)

             country  year         gpd
0             Sweden  1800   34.327801
1     United Kingdom  1800  176.302994
2      United States  1800   18.091101
3             Sweden  1801   34.250599
4     United Kingdom  1801  178.988007
...              ...   ...         ...
9048      Uzbekistan  2023   32.514054
9049         Vanuatu  2023   41.858286
9050       Venezuela  2023  146.261879
9051         Vietnam  2023   34.370721
9052           Yemen  2023   81.205318

[9053 rows x 3 columns]


In [ ]:




app = Dash( suppress_callback_exceptions=True)

app.config.suppress_callback_exceptions = True
app.config.prevent_initial_callbacks = 'initial_duplicate'

#---------------------------------------------------------------------------------------------------------------------------------------
                                #UTILS PRIMEIRA PAGINA

#lista dos paises desejados para o primeiro grafico
filtro_paises_graph1 = [
    "Brazil", "China", "United States", "United Kingdom", "Venezuela (Bolivarian Republic of)",
    "Russian Federation", "Ukraine", "Australia", "South Africa", "Qatar"
] 

df_gdppc_graph1 = df_gdppc[df_gdppc['country'].isin(filtro_paises_graph1)] #filtro dados por paises

# Pegando o range dos anos e convertendo para int
ano_min = int(df_gdppc_graph1['year'].min())
ano_max = int(df_gdppc_graph1['year'].max())

filtro_graph1 = list(df_gdppc_graph1['country'].unique())
filtro_graph1.append("All"),
                                #FIM UTILS PRIMEIRA PAGINA

#---------------------------------------------------------------------------------------------------------------------------------------

                                #UTILS SEGUNDA PAGINA
df_gdppc_graph2 = df_gdppc
gdppc_usa = df_gdppc_graph2.loc[df_gdppc_graph2['country'] == 'United States', 'gdppc'].values[0]
df_gdppc_graph2["percent_of_usa"] = (df_gdppc_graph2["gdppc"]/ gdppc_usa)

                                #FIM UTILS SEGUNDA PAGINA

#---------------------------------------------------------------------------------------------------------------------------------------

                                #UTILS TERCEIRA PAGINA

df_gpd_graph3 = df_gdppc.merge(df_gpd, on=['year', 'country'], how='inner')
lista_paises_graph3 = df_gpd_graph3['country'].unique()

df_gpd_graph3['gdppc'] = pd.to_numeric(df_gpd_graph3['gdppc'], errors='coerce')
df_gpd_graph3['gpd'] = pd.to_numeric(df_gpd_graph3['gpd'], errors='coerce')
df_gpd_graph3.to_sql('final', con=engine, if_exists='replace', index=False) #salvei no banco de dados o arquivo "final" com as tabelas de gdppc, gpd, percent_of_usa, pop, year, region, country, countrycode

df_gpd_graph3['gpd'] = df_gpd_graph3['gpd'] * 100

                                #FIM UTILS TERCEIRA PAGINA
#---------------------------------------------------------------------------------------------------------------------------------------


                                #SETANDO GRAFICOS E LAYOUT QUE POSSIBILITA TRANSIÇÃO DE PAGINAS

fig_pagina1 = px.line(df_gdppc_graph1, x="year", y="gdppc", color="country") #Grafico para a primeira
fig_pagina2 = px.bar(df_gdppc_graph2, x="countrycode", y="percent_of_usa") #Grafico para a segunda pagina
fig_pagina3 = px.line(df_gdppc_graph1, x="year", y="gdppc") #Grafico para a terceira pagina

app.layout = html.Div([
    dcc.Location(id='url', refresh=False), #para monitorar a url atual
    html.Nav([
        dcc.Link('Página 1 (PIB per Capita) |', href='/'),
        dcc.Link('| Página 2 (Comparação PIB com % USA) |', href='/pagina2'),
        dcc.Link('| Página 3 (Comparação PIB com % Brasil) ', href='/pagina3'),
    ], style={'margin-bottom' : '20px', 'display' : 'flex', 'justify-content' : 'space-between', 'font-family':'Poppins, sans-serif'}),
    html.Div(id='page-content', style={'font-family':'Poppins, sans-serif', 'backgrpund-color':'black'})
])
#---------------------------------------------------------------------------------------------------------------------------------------

                                #LAYOUT
#layout pagina 1
page1_layout=html.Div([
    html.H1(children='PIB per Capita dos Países selecionados', style={'font-family':'Poppins, sans-serif'}),
    html.H3(children='Filtre por país', style={'font-family':'Poppins, sans-serif'}),
        dcc.Dropdown(
         options=[{'label': i, 'value': i} for i in filtro_graph1],
         id='lista_paises',
         value="All",
         style={'width': '50%', 'font-family':'Poppins, sans-serif'}
         
    ),

       html.H3(children='Filtre por ano', style={'font-family':'Poppins, sans-serif'}),
    
    dcc.RangeSlider(
    id='filtro_anos',
    min=ano_min,
    max=ano_max,
    step=2,
    marks={
        ano_min: str(ano_min),
        ano_max: str(ano_max)
    },
    value=[ano_min, ano_max]  # Valor inicial todos os anos
    ),
    html.Div([
        html.Div(id='ano_min', children=f"Min: {ano_min}", style={'display': 'inline-block', 'padding': '10px','font-family':'Poppins, sans-serif'}),
        html.Div(id='ano_max', children=f"Max: {ano_max}", style={'display': 'inline-block', 'padding': '10px', 'font-family':'Poppins, sans-serif'}),
    ]),
    
    dcc.Graph(
        id='graph_gdppc',
        figure=fig_pagina1
    ),

 
]),


#layout pagina 2
page2_layout=html.Div([
    html.H1(children='PIB per Capita em comparação com porcentagem do USA'),
    html.H3(children='Filtros'),
        html.H3("Selecione o Ano"),
    dcc.Dropdown(
        id='filtro_ano_unico',
        options=[{'label': str(ano), 'value': ano} for ano in range(ano_min, ano_max + 1)],
        value=ano_max,  # Colocando o maior valor dentro de ano como padrão para o filtro
        clearable=False,
        style={'width': '50%'}
    ),
     

    dcc.Graph(
        id='graph_gdppc_%',
        figure=fig_pagina2
    )
]),

#layout pagina 3

page3_layout = html.Div([
    html.H1("Evolução do PIB per capita e PIB Total"),
    html.H3("Selecione um País"),
    dcc.Dropdown(
        id='filtro_paises_pagina3',
        options=[{'label': pais, 'value': pais} for pais in lista_paises_graph3],  
        style={'width': '75%'},
        value=lista_paises_graph3[0],  # Define o primeiro país como padrão
        multi=False  # Permite selecionar apenas um país por vez
    ),

    dcc.Graph(id='graph_gdppc_gpd_linhas') ,

    
    
    
])



                                #FIM DOS LAYOUTS

#---------------------------------------------------------------------------------------------------------------------------------------
                                #CALLBACK PARA MUDANÇAS DE PAGINAS

@app.callback(
    Output('page-content', 'children'),
    [Input('url', 'pathname')]
)
def display_page(pathname):
    if pathname == '/pagina2':
        return page2_layout 
    elif pathname == '/pagina3':
        return page3_layout
    else:
        return page1_layout  # Página inicial

                                #FIM CALLBACK PARA MUDANÇAS DE PAGINAS

#---------------------------------------------------------------------------------------------------------------------------------------

                                #CALLBACK PRIMEIRA PAGINA

@app.callback(
    [Output('graph_gdppc', 'figure', allow_duplicate=True),
     Output('ano_min', 'children'),
     Output('ano_max', 'children')],
    [Input('lista_paises', 'value'),
     Input('filtro_anos', 'value')],
    [State('url', 'pathname')]
)
def filtro_pagina1 (pais, anos, pathname):
    if pathname != '/':
        raise dash.exceptions.PreventUpdate

    # Filtro do slider por anos
    ano_min_value = anos[0]
    ano_max_value = anos[1]

    df_gdppc_graph1_filtrado = df_gdppc_graph1[
        (df_gdppc_graph1['year'] >= ano_min_value) & 
        (df_gdppc_graph1['year'] <= ano_max_value)
    ]
    #Filtro por paises
    if pais != "All":
        df_gdppc_graph1_filtrado = df_gdppc_graph1_filtrado[df_gdppc_graph1_filtrado['country'] == pais]

    # Cria o gráfico
    fig = px.line(df_gdppc_graph1_filtrado, x="year", y="gdppc", color="country", title=f'PIB per capita de {pais}')

    # Atualiza os valores do slider
    return fig, f"Min: {ano_min_value}", f"Max: {ano_max_value}"



    

                                #FIM CALLBACK PRIMEIRA PAGINA

#---------------------------------------------------------------------------------------------------------------------------------------
    
                                #CALLBACK SEGUNDA PAGINA

@app.callback(
    Output('graph_gdppc_%', 'figure', allow_duplicate=True),
    [Input('filtro_ano_unico', 'value')],
    [State('url', 'pathname')]
)
def filtro_pagina2(selected_year, pathname):
    if pathname != '/pagina2':
        raise dash.exceptions.PreventUpdate

    df_gdppc_graph2_filtrado = df_gdppc_graph2[df_gdppc_graph2['year'] == selected_year]

    # Obtém o PIB dos EUA
    gdppc_usa = df_gdppc_graph2_filtrado.loc[df_gdppc_graph2_filtrado['country'] == 'United States', 'gdppc'].values
    if len(gdppc_usa) > 0:
        gdppc_usa = gdppc_usa[0]
        df_gdppc_graph2_filtrado['percent_of_usa'] = (df_gdppc_graph2_filtrado['gdppc'] / gdppc_usa) * 100

        # Definir cores personalizadas (USA = Vermelho, Outros = Azul)
        def definir_cor(pais):
                if pais == 'United States':
                    return 'USA' #Vermelho
                return 'Outros' #Azul
        df_gdppc_graph2_filtrado['color'] = df_gdppc_graph2_filtrado['country'].apply(definir_cor)

        # Criar gráfico de barras com cores personalizadas
        fig = px.bar(df_gdppc_graph2_filtrado, x='countrycode', y='percent_of_usa',  
                     title=f'PIB % comparado ao USA ({selected_year})',
                     color='color', 
                     color_discrete_map={'USA': '#BD0000', 'Outros': '#2596BE'})  # Mapeando cores

    else:
        fig = px.bar(title='Dados do USA não disponíveis para o ano selecionado')

    return fig

                                #FIM DOS CALLBACK SEGUNDA PAGINA

#---------------------------------------------------------------------------------------------------------------------------------------

                                #CALLBACK TERCEIRA PAGINA

@app.callback(
    Output('graph_gdppc_gpd_linhas', 'figure'),  
    [Input('filtro_paises_pagina3', 'value')],  
    [State('url', 'pathname')]  
)
def filtro_pagina3(pais_selecionado, pathname):
    if pathname != '/pagina3':
        raise dash.exceptions.PreventUpdate

    # Garantia de que há um país selecionado
    if not pais_selecionado:
        pais_selecionado = lista_paises_graph3[0]  # Define um padrão caso nenhum seja escolhido

    # Filtrar DataFrame pelo país selecionado
    df_filtrado = df_gpd_graph3[df_gpd_graph3['country'] == pais_selecionado]

    # Criar gráfico de linhas
    fig = px.line(df_filtrado, x='year', y=['gdppc', 'gpd'], 
                  title=f'Evolução do PIB per Capita e PIB Total - {pais_selecionado}',
                  labels={'value': 'Valor', 'variable': 'Métrica'},
                  markers=True,
                  color_discrete_map={'gdppc':'#2596BE', 'gpd':'#BD0000'})  
    return fig


if __name__ == '__main__':
    app.run(debug=True)


C:\Users\lilri\AppData\Local\Temp\ipykernel_6720\2727963351.py:228: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\lilri\AppData\Local\Temp\ipykernel_6720\2727963351.py:235: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
print(df_gdppc)